In [1]:
import requests 
import bs4
import zipfile
import re


url = 'https://unsplash.com/'
# url = 'https://collections.louvre.fr/en/'
# url = 'https://scryfall.com/sets/rvr'


def get_jpg(src_url, i, zpf):
    print(src_url)
    response_jpg = requests.get(src_url)    
    if response_jpg.status_code != 200:
        print('Невалидная ссылка, код ответа:', response_jpg.status_code)
    else:
        if not response_jpg.headers['Content-Type'].startswith('image/'):
            print('Не картинка')
        else:
            # Решил отсечь часть изображений по размеру файла, с unsplash тянул аватарки
            size = int(response_jpg.headers.get('Content-Length', 0))
            if size < 5*1024:
                print('Изображение маленькое, игнорируем')
            else: 
                file_name = f'image{i}.jpg'
                zpf.writestr(file_name, response_jpg.content)

def get_zip(url):
    # на ресурсах отличается формат хранения, не все содержат домен
    match = re.match(r"^https?://[^\/]+", url)
    base_url = match.group(0)

    response = requests.get(url)
    soup = bs4.BeautifulSoup(response.text, 'html.parser')
    # собираем все src, до которых можем дотянуться, игнорируя их происхождение.
    # от разумного подхода к работе с тегами отказался в пользу универсальности
    src_urls = set()
    for tag in soup.find_all():
        if tag.has_attr('src'):
            src_urls.add(tag['src'])
            
    with zipfile.ZipFile('images_try2.zip', 'w') as zpf:
        for i, src_url in enumerate(src_urls):
            if '.svg' in src_url:
                    print('Игнорируем векторную графику')
            else:
                if 'http' in src_url:
                    get_jpg(src_url, i, zpf)
                elif src_url[1] == '/':
                    get_jpg(base_url+src_url, i, zpf)
                    


get_zip(url)
        


https://images.unsplash.com/photo-1695751240180-6194e4d152a8?ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxlZGl0b3JpYWwtZmVlZHw1fHx8ZW58MHx8fHx8&w=1000&q=80
https://images.unsplash.com/photo-1695782098642-b71604fcb72a?ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxlZGl0b3JpYWwtZmVlZHw0fHx8ZW58MHx8fHx8&w=1000&q=80
https://images.unsplash.com/profile-1667399841090-6cbac9788d33image?auto=format&fit=crop&w=16&h=16&q=60&crop=faces&bg=fff
Изображение маленькое, игнорируем
https://images.unsplash.com/profile-1690060216823-f69256563b2cimage?auto=format&fit=crop&w=32&h=32&q=60&crop=faces&bg=fff
Изображение маленькое, игнорируем
https://images.unsplash.com/photo-1682687221363-72518513620e?ixlib=rb-4.0.3&ixid=M3wxMjA3fDF8MHxlZGl0b3JpYWwtZmVlZHwxfHx8ZW58MHx8fHx8&w=1000&q=80
https://images.unsplash.com/photo-1696506473657-b3edcadfe08c?ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxlZGl0b3JpYWwtZmVlZHw4fHx8ZW58MHx8fHx8&w=1000&q=80
https://images.unsplash.com/photo-1696416748833-0407ebea6047?ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxlZGl0b3J